# ABS Quarterly Producer Prices Index 6427

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
    get_ABS_meta_and_data,
    find_id,
)
from plotting import (
    set_chart_dir,
    clear_chart_dir,
    plot_growth_finalise,
    calc_growth,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in the notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "6427"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

# do the data capture and extraction to a DataFrame
abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)

meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plot

### Set-up some constants we will use across all plots

In [3]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Headline PPI for Australia

In [4]:
quarterly_phrase = "Percentage change from previous quarter"
annual_phrase = "Percentage change from corresponding quarter of previous year"
final = "Final ;  Total (Source)"

In [5]:
get = {
    (final, "1"): "Final Demand",
    ("Coal mining", "11"): "Coal mining",
    ("Manufacturing division", "12"): "Manufacturing",
    # ('Building construction Australia', '17'): 'Building construction Australia',
    # ('Road freight transport ', '21'): ' Road freight transport ',
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    identifiers = []
    for phrase in annual_phrase, quarterly_phrase:
        terms = {
            table: table_col,
            select_phrase: did_col,
            phrase: did_col,
        }
        ident, units = find_id(meta, terms, verbose=False)
        identifiers.append(ident)

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        )
        plot_growth_finalise(
            data[identifiers[0]],
            data[identifiers[1]],
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter="Australia: Producer Price Index. Original series",
            show=SHOW,
        )

### Lesser PPIs - no ABS pre-calculated percentages

In [6]:
get = {
    ("30 Building construction Australia ;", "17"): "Building construction Australia",
    ("Road freight transport ", "21"): "Road freight transport",
    ("Employment services", "25"): "Employment services",
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    terms = {
        table: table_col,
        select_phrase: did_col,
        "Index Number": did_col,
    }
    ident, units = find_id(meta, terms, verbose=False)
    growth = calc_growth(abs_dict[table][ident])

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        )
        plot_growth_finalise(
            *growth,
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter=f"Australia: Producer Price Index. Original series",
            show=SHOW
        )

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Apr 29 2023 14:13:41

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.12.1

pandas    : 2.0.1
matplotlib: 3.7.1
sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]

Watermark: 2.3.1



In [8]:
print("Finished")

Finished
